In [1]:
import pandas as pd
from math import sqrt
import numpy as np

In [109]:
books_df = pd.read_csv("BX-Books.csv", delimiter = ";", encoding = "latin-1", error_bad_lines = False)
ratings_df = pd.read_csv("BX-Book-Ratings.csv", delimiter = ";", encoding = "latin-1", error_bad_lines = False)
print(books_df.info())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13104\3051949703.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books_df = pd.read_csv("BX-Books.csv", delimiter = ";", encoding = "latin-1", error_bad_lines = False)
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None


In [110]:
userInput = [{'Title':'The Mummies of Urumchi', 'Rating': 0},
            {'Title':'The Takeover', 'Rating': 0},
            {'Title':'Decision in Normandy', 'Rating': 7},
            {'Title':'Go to Your Room', 'Rating': 6},
            {'Title':'La Perle', 'Rating': 9}]
        
inputBooks = pd.DataFrame(userInput)
print(inputBooks)

                    Title  Rating
0  The Mummies of Urumchi       0
1            The Takeover       0
2    Decision in Normandy       7
3         Go to Your Room       6
4                La Perle       9


In [111]:
inputID = books_df[books_df["Book-Title"].isin(inputBooks["Title"].tolist())]
inputBooks = pd.merge(inputID, inputBooks, left_index = True, right_index = True)
inputBooks = inputBooks[['ISBN','Title','Rating']]
print(inputBooks)

         ISBN                 Title  Rating
2  0060973129  Decision in Normandy       7
4  0393045218              La Perle       9


In [112]:
userSubset = ratings_df[ratings_df['ISBN'].isin(inputBooks['ISBN'].tolist())]
print(userSubset.groupby('ISBN').count())

            User-ID  Book-Rating
ISBN                            
0060973129        3            3
0393045218        1            1


In [117]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['ISBN'])

def take_5_elem(x):
    # print (len(x[1]))
    return len(x[1])
    
userSubsetGroup = sorted(userSubsetGroup, key = take_5_elem, reverse = True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])

[('0060973129',         User-ID        ISBN  Book-Rating
9564          8  0060973129            0
16919      2954  0060973129            8
629379   152827  0060973129            7), ('0393045218',       User-ID        ISBN  Book-Rating
9566        8  0393045218            0)]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13104\2616816697.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key = take_5_elem, reverse = True)


In [119]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:

    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='ISBN')
    inputBooks = inputBooks.sort_values(by='ISBN')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    temp_df = inputBooks[inputBooks['ISBN'].isin(group['ISBN'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['Rating'].tolist()

    #Let's also put the current user group reviews in a list format
    tempGroupList = group['Book-Rating'].tolist()
    
    #Now let's calculate the pearson correlation between two users, so called, x and y manually (check the formula from week 7 slide)
    Sxx = sum([pow(i,2) for i in tempRatingList]) - pow(sum(tempRatingList),2)/int(nRatings)
    Syy = sum([pow(i,2) for i in tempGroupList]) - pow(sum(tempGroupList),2)/int(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/int(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [140]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['ISBN'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex        ISBN
0        -0.993399  0060973129
1         0.000000  0393045218


In [141]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

   similarityIndex        ISBN
1         0.000000  0393045218
0        -0.993399  0060973129


In [163]:
topUsersRating=topUsers.merge(ratings_df, left_on='ISBN', right_on='ISBN', how='inner')
print(topUsersRating.head(100))

   similarityIndex        ISBN  User-ID  Book-Rating
0         0.000000  0393045218        8            0
1        -0.993399  0060973129        8            0
2        -0.993399  0060973129     2954            8
3        -0.993399  0060973129   152827            7


In [164]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Book-Rating']
print(topUsersRating.head())

   similarityIndex        ISBN  User-ID  Book-Rating  weightedRating
0         0.000000  0393045218        8            0        0.000000
1        -0.993399  0060973129        8            0       -0.000000
2        -0.993399  0060973129     2954            8       -7.947194
3        -0.993399  0060973129   152827            7       -6.953795


In [166]:
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

            sum_similarityIndex  sum_weightedRating
ISBN                                               
0060973129            -2.980198          -14.900989
0393045218             0.000000            0.000000


In [167]:
recommendation_df = pd.DataFrame()

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
print(recommendation_df.head(10))

            weighted average recommendation score        ISBN
ISBN                                                         
0060973129                                    5.0  0060973129
0393045218                                    NaN  0393045218


In [168]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df)

            weighted average recommendation score        ISBN
ISBN                                                         
0060973129                                    5.0  0060973129
0393045218                                    NaN  0393045218


In [169]:
recommended_books = books_df.loc[books_df["ISBN"].isin(recommendation_df["ISBN"])]

#we don't want to recommend the same movie
recommended_books = recommended_books.loc[~recommended_books.ISBN.isin(userSubset["ISBN"])]

print(recommended_books)

Empty DataFrame
Columns: [ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher, Image-URL-S, Image-URL-M, Image-URL-L]
Index: []
